In [1]:
import torch
from torchsummary import summary
import torchgadgets as tg
from run import *

/home/user/denninge/anaconda3/envs/vision_lab/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
exp_name = 'svhn_gan'
run_name = 'test_small'

trainer = Trainer(exp_name, run_name)

Using downloaded and verified file: ./data/svhn/train_32x32.mat
Using downloaded and verified file: ./data/svhn/test_32x32.mat


### Generator

In [3]:
trainer.gan.generator

Generator(
  (latent_emb): Sequential(
    (0): Linear(in_features=100, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
    (3): Reshape()
  )
  (label_emb): Sequential(
    (0): Linear(in_features=10, out_features=1024, bias=True)
    (1): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.2)
    (3): Reshape()
  )
  (model): Sequential(
    (0): ConvTransposeBlock(
      (block): Sequential(
        (0): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2)
      )
    )
    (1): ConvTransposeBlock(
      (block): Sequential(
        (0): ConvTranspose2d(64, 32, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, m

In [4]:
if trainer.conditional:
    inp = trainer.config['model']['latent_dim']+trainer.config['model']['num_labels']
else:
    inp = trainer.config['model']['latent_dim']
print("input Dim :", inp)
summary(trainer.gan.generator.cuda(), (1024,inp))

input Dim : 110


RuntimeError: mat1 and mat2 shapes cannot be multiplied (2048x110 and 100x1024)

### Discriminator

In [5]:
trainer.gan.discriminator

Discriminator(
  (img_emb): Sequential(
    (0): ConvBlock(
      (block): Sequential(
        (0): Conv2d(3, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2)
      )
    )
  )
  (label_emb): Sequential(
    (0): ConvBlock(
      (block): Sequential(
        (0): Conv2d(10, 16, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2)
      )
    )
  )
  (model): Sequential(
    (0): ConvBlock(
      (block): Sequential(
        (0): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): LeakyReLU(negative_slope=0.2)
      )
    )
    (1): ConvBlock(
      (block): Sequential(
        (0): Conv2d(64, 128,

In [ ]:
inp_dim = trainer.config['model']['discriminator']['input_size']
if trainer.conditional:
    inp_dim[0] += trainer.config['model']['num_labels']
print("input Dim :", inp_dim)
summary(trainer.gan.discriminator.cuda(), tuple(inp_dim))


input Dim : [13, 32, 32]
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 16, 16]           6,688
       BatchNorm2d-2           [-1, 32, 16, 16]              64
         LeakyReLU-3           [-1, 32, 16, 16]               0
         ConvBlock-4           [-1, 32, 16, 16]               0
            Conv2d-5             [-1, 64, 8, 8]          32,832
       BatchNorm2d-6             [-1, 64, 8, 8]             128
         LeakyReLU-7             [-1, 64, 8, 8]               0
         ConvBlock-8             [-1, 64, 8, 8]               0
            Conv2d-9            [-1, 128, 4, 4]         131,200
      BatchNorm2d-10            [-1, 128, 4, 4]             256
        LeakyReLU-11            [-1, 128, 4, 4]               0
        ConvBlock-12            [-1, 128, 4, 4]               0
          Flatten-13                 [-1, 2048]               0
           Lin